<a href="https://colab.research.google.com/github/sberdine77/LSTM-aws-spot-price/blob/main/notebook/LSTM_aws_spot_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Necessary instalations
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install tensorflow-macos
!{sys.executable} -m pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 3.8 MB/s eta 0:00:0000:0100:01
  Using cached scipy-1.10.1-cp39-cp39-macosx_12_0_arm64.whl (28.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.0 MB/s eta 0:00:00a 0:00:01


In [4]:
#!{sys.executable} -m pip install numpy --upgrade
#Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [29]:
tf.random.set_seed(7)
df = pd.read_json('../data/spot-price-history.json')
df = df[['Timestamp', 'SpotPrice']]
df['Timestamp'] = df['Timestamp'].astype(str)
df['revenue'] = df['revenue'].apply(lambda x: float(x.split()[0].replace(' ', '')))

dataset = df.values
print(dataset)
dataset = dataset.astype('float64')
# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)
print(dataset)

[['2023-01-31 21:07:29+00:00' 0.0805]
 ['2023-01-31 21:07:29+00:00' 0.1205]
 ['2023-01-31 21:07:29+00:00' 0.0205]
 ...
 ['2022-11-29 01:51:45+00:00' 0.119]
 ['2022-11-29 01:51:45+00:00' 0.019]
 ['2022-11-29 01:51:45+00:00' 0.019]]


ValueError: could not convert string to float: '2023-01-31 21:07:29+00:00'